# Projeto Semestral de Ciência de Dados - Classificação de artigos científicos do site Arxiv

### Etapa 1 - Coleta de Dados

**Instalação de bibliotecas**

In [ ]:
!pip install arxiv
!pip install pandas
!pip install sklearn
!pip install tensorflow  # Para usar modelos pré-treinados
!pip install streamlit  # Para criar o app (a instalação é só para testes locais)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=2bab370b32d991bfb0ffcf96c414e03ed7e13bb5ba2dddd2cf9b833cf00abce4
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

**Coleta de Dados**

In [ ]:
import arxiv
import pandas as pd

# Definindo as categorias para coleta (Exemplo: IA e aprendizado de máquina)
categories = ["cs.AI", "stat.ML", "cs.LG", "cs.CR"]
num_articles = 200  # Número de artigos por categoria

# Função para coletar artigos de uma categoria
def fetch_arxiv_data(category, num_articles):
    search = arxiv.Search(
        query=f"cat:{category}",
        max_results=num_articles,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    data = []
    for result in search.results():
        data.append({
            'title': result.title,
            'summary': result.summary,
            'category': category
        })
    return pd.DataFrame(data)

# Coletando dados de todas as categorias
df_list = [fetch_arxiv_data(cat, num_articles) for cat in categories]
df = pd.concat(df_list, ignore_index=True)

# Salvando os dados em um arquivo CSV (opcional)
df.to_csv("arxiv_data.csv", index=False)
print(f"Coletamos {len(df)} artigos.")
df.head()


<ipython-input-4-fbc60e3758be>:16: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Coletamos 1600 artigos.


,title,summary,category
0,Scaling Properties of Diffusion Models for Per...,"In this paper, we argue that iterative computa...",cs.AI
1,GaussianAnything: Interactive Point Cloud Late...,While 3D content generation has advanced signi...,cs.AI
2,Learning with Less: Knowledge Distillation fro...,"In real-world NLP applications, Large Language...",cs.AI
3,LLMPhy: Complex Physical Reasoning Using Large...,Physical reasoning is an important skill neede...,cs.AI
4,Leonardo vindicated: Pythagorean trees for min...,Trees continue to fascinate with their natural...,cs.AI


**Pré processamento dos Dados**

In [ ]:
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Limpeza de texto
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove espaços extras
    text = re.sub(r'\W', ' ', text)   # Remove caracteres especiais
    text = text.lower()               # Converte para minúsculas
    return text

# Aplicando limpeza no texto
df['cleaned_summary'] = df['summary'].apply(clean_text)

# Convertendo as classes em números
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# Separando X e y
X = df['cleaned_summary']
y = df['category_encoded']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Pré-processamento concluído.")


Pré-processamento concluído.


### Etapa 2 - Processamento ML

**Extração de Features com Embeddings**

In [ ]:
import tensorflow_hub as hub
import numpy as np

# Carregando o Universal Sentence Encoder para embeddings
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Função para converter textos em embeddings
def get_embeddings(texts):
    return np.array([embed([text])[0].numpy() for text in texts])

# Convertendo os textos em embeddings
X_train_embeddings = get_embeddings(X_train)
X_test_embeddings = get_embeddings(X_test)

print("Embeddings criados com sucesso.")


Embeddings criados com sucesso.


**Treinamento do Modelo**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Treinando o modelo
clf = RandomForestClassifier()
clf.fit(X_train_embeddings, y_train)

# Avaliando o modelo
y_pred = clf.predict(X_test_embeddings)
accuracy = accuracy_score(y_test, y_pred)

print(f"Acurácia do modelo: {accuracy * 100:.2f}%")


Acurácia do modelo: 80.31%


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Treinando o modelo
clf = DecisionTreeClassifier()
clf.fit(X_train_embeddings, y_train)

# Avaliando o modelo
y_pred = clf.predict(X_test_embeddings)
accuracy = accuracy_score(y_test, y_pred)

print(f"Acurácia do modelo: {accuracy * 100:.2f}%")


Acurácia do modelo: 69.69%


### Etapa 3

**Criação do Streamlit**

In [ ]:
import joblib
joblib.dump(clf, 'modelo_treinado.pkl')

['modelo_treinado.pkl']